# 19. Math
#### By Kwan Yin Andrew Chau

In [1]:
import nbfinder
from loader import g
from loader import __
from loader import sydneyVertexId
from loader import hkVertexId

## math()

__Vertex found at the start of the query is not used by the math step__

In [2]:
y = g.V().limit(1).math('(100/2) * 0.5 + 2 - 1').next()
assert y == 26
y

26.0

__Math on the injected value__

In [3]:
y1 = g.inject(100).as_('x').math('x/2').next()
assert y1 == 50
y1

50.0

__We can use _ instead of selecting the previous step__

In [4]:
y2 = g.inject(100).math('_/2').next()
assert y1 == y2
y2

50.0

__modulo__

In [5]:
y = g.V().limit(1).math('103 % 50').next()
assert y == 3
y

3.0

__power__

In [6]:
y = g.V().limit(1).math('2^5').next()
assert y == 32
y

32.0

__absolute value__

In [7]:
x = -9
y = g.V().limit(1).math('abs(' + str(x) + ')').next()
assert y == -x
y

9.0

__cosine and arc cosine__

In [8]:
x = -1
y = g.V().limit(1).math('cos(acos(' + str(x) + '))').next()
assert y == x
y

-1.0

__sine and arc sine__

In [9]:
x = 1
y = g.V().limit(1).math('sin(asin(' + str(x) + '))').next()
assert x == y
y

1.0

__tangent and arc tangent__

In [10]:
x = 0
y = g.V().limit(1).math('tan(atan(' + str(x) + '))').next()
assert x == y
y

0.0

__square root__

In [11]:
x = 16
y = g.V().limit(1).math('sqrt(' + str(x) + ')').next()
assert y == 4
y

4.0

__cube root__

In [12]:
y = g.V().limit(1).math('cbrt(27)').next()
assert y == 3
y

3.0

__ceil__

In [13]:
y = g.V().limit(1).math('ceil(1.3)').next()
assert y == 2
y

2.0

__floor__

In [14]:
y = g.V().limit(1).math('floor(1.7)').next()
assert y == 1
y

1.0

__natural log and euler__

In [15]:
y = g.V().limit(1).math('exp(log(1))').next()
assert y == 1
y

1.0

__log base 10__

In [16]:
y = g.V().limit(1).math('log10(1000)').next()
assert y == 3
y

3.0

__log base 2__

In [17]:
y = g.V().limit(1).math('log2(16)').next()
assert y == 4
y

4.0

__Use signum() to return 0 if x = 0, 1 if x > 0, -1 if x < 0__

In [18]:
y = g.V().limit(1).math('signum(-12)').next()
assert y == -1
y

-1.0

In [19]:
y = g.V().limit(1).math('signum(0)').next()
assert y == 0
y

0.0

In [20]:
y = g.V().limit(1).math('signum(2112)').next()
assert y == 1
y

1.0

__Add the total number of runways in HK and Sydney together__

In [21]:
x1 = g.V(hkVertexId).values('runways').next()
x2 = g.V(sydneyVertexId).values('runways').next()
y = g.V(hkVertexId).as_('hk').out().hasId(sydneyVertexId).as_('syd').math('hk + syd').by('runways').next()
assert y == x1 + x2
y

5.0

__We apply project() here too to show the conversion of the longest runway in HK from feet and meters__

In [22]:
g.V(hkVertexId).project(
    'Longest runway in','feet','meters'
).by('desc').by('longest').by(
    __.values('longest').math('ceil(_ * 0.3048)')
).next()

{'Longest runway in': 'Hong Kong - Chek Lap Kok International Airport',
 'feet': 12467,
 'meters': 3800.0}

Use math() to add Sydney's incoming and outgoing routes together

In [23]:
g.V(sydneyVertexId).project(
    'in', 'out'
).by(__.in_('route').count()).by(__.out('route').count()).math('in + out').next()

188.0

__Using signum() in choose() and options()__

In [24]:
g.V().hasLabel('airport').sample(12).project(
    'IATA' ,'city', 'position'
).by('code').by('city').by(
    __.choose(
        __.values('lon').math('signum(_)')
    ).option(
        -1.0, __.constant('West')
    ).option(
        0.0, __.constant('On the meridian')
    ).option(
        1.0, __.constant('East')
    )
).fold().next()

[{'IATA': 'TBU', 'city': "Nuku'alofa", 'position': 'West'},
 {'IATA': 'BRW', 'city': 'Barrow', 'position': 'West'},
 {'IATA': 'NAJ', 'city': 'Nakhchivan', 'position': 'East'},
 {'IATA': 'BLQ', 'city': 'Bologna', 'position': 'East'},
 {'IATA': 'ABX', 'city': 'Albury', 'position': 'East'},
 {'IATA': 'RMF', 'city': 'Marsa Alam', 'position': 'East'},
 {'IATA': 'DNK', 'city': 'Dnipropetrovsk', 'position': 'East'},
 {'IATA': 'AUH', 'city': 'Abu Dhabi', 'position': 'East'},
 {'IATA': 'MDQ', 'city': 'Mar del Plata', 'position': 'West'},
 {'IATA': 'PIR', 'city': 'Pierre', 'position': 'West'},
 {'IATA': 'MEC', 'city': 'Manta', 'position': 'West'},
 {'IATA': 'CFG', 'city': 'Cienfuegos', 'position': 'West'}]

__Calculate the standard deviation of runways in an airport__

In [25]:
mean = g.V().hasLabel('airport').values('runways').mean().next()
mean

1.4309425014819206

In [26]:
count = g.V().hasLabel('airport').count().next()
count

3374

In [27]:
# σ = sqrt((1/count) * Σ((Xi - mean)^2), where i=1
g.withSideEffect(
    "mean", mean
).withSideEffect(
    "count", count
).V().hasLabel('airport').values(
    'runways'
).math('(_ - mean)^2').sum().math('_ / count').math('sqrt(_)').next()

0.7510927827902378